# Homework Tasks

Read about difference between GPT-3.5 and GPT-4.

Read about metrics for generarive NLP.

**Advanced**: Generative models are usually very big. Read about model quantization. That may help with inference of big models such as GPT.

**Theory** (5 points): Google form questions.

**Practical task** (10 points):
1. Choose one:
    * Finetune transformer model for summarization on https://huggingface.co/datasets/samsum.
    * Finetune transformer model for translation on dataset of your choice.
2. Experiment with different prompts.
2. Based on a task you choose, choose a few metrics that are used in generative NLP (BLEU, ROUGE etc), test your finetune models using them, describe their pros and cons relative to the generations your model makes.

3. If you want, you can try use LoRA or prefix tuning for finetuning the model.

## Imports

In [1]:
!pip install -q transformers datasets evaluate peft py7zr rouge_score bert_score bitsandbytes accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

import nltk
nltk.download('punkt')
import evaluate
import torch

import torch.nn as nn
import bitsandbytes as bnb
import pandas as pd
import numpy as np

from evaluate import evaluator
from transformers import AutoTokenizer, GenerationConfig, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AdamW
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import notebook_login
from peft import PeftConfig, PeftModel

notebook_login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 M

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
class CFG:
    # model_path = 'facebook/bart-base'
    lora_model_hub_id = 'NochnoyRitzar/t5-small-summarization-samsum'
    model_path = 't5-small'
    dataset_path = 'samsum'
    max_text_length = 512
    max_summary_length = 50

    # Training parameters
    fp16 = True
    learning_rate = 1e-4
    # weight_decay = 0.01
    warmup_steps = 500
    num_epochs = 3
    per_device_batch_size = 16



## EDA

In [3]:
dataset = load_dataset(CFG.dataset_path)
print(dataset)

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


SAMSun dataset already has training, validation and testing splits

Take a look at text structure in this dataset

In [4]:
for split in ['train', 'validation', 'test']:
    print(f'--------------------   {split.upper()} Split  --------------------')
    print(dataset[split][200]['dialogue'])
    print(f"\n{dataset[split][200]['summary']}\n\n")

--------------------   TRAIN Split  --------------------
Rachel: HAPPY NEW YEAAAAAAR!! ヽ(^o^)丿
Rachel: from me & Tom :)
Jack: Thanks!!
Miranda: same to you
Miranda: :)
Rachel: May you get exactly what you are expecting in the coming year!!!
Miranda: :*
Jack: How's your party?
Rachel: %) %) %) 
Jack: Hahahah
Jack: I see
Rachel: the best party eveeeeeeeer
Rachel: wish you were here!!!

Rachel wishes a happy new year from her and Tom to Miranda and Jack. Rachel is having the best time but misses Miranda and Jack.


--------------------   VALIDATION Split  --------------------
Andy: are you ok with 6PM at Mammals?
Barbara: I won't manage, but you can start without me
Peter: too early for me as well
Paul: by 8PM I won't make it
Simone: haha, seems I'm the only one who's ok with it, let's move it to 8PM
Andy: ok, as you prefer, but free drinks end at 7.30
Barbara: so I would say everybody comes when they can
Andy: right, I'll be there at 6.30, but I may be tipsy at 8
Simone: tipsy? drunk as 

Dataset has a dialogue and it's summary as training data

## Data processing

In [5]:
dataset = load_dataset(CFG.dataset_path)

# Create tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Find optimal sequence length for padding and truncation

In [6]:
concat_datasets = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])

tokenized_inputs = concat_datasets.map(lambda x: tokenizer(x['dialogue'], truncation=True), batched=True)['input_ids']
tokenized_inputs_len = [len(seq) for seq in tokenized_inputs]
max_source_length = int(np.percentile(tokenized_inputs_len, 90))
print(f"Max source length: {max_source_length}")


tokenized_targets = concat_datasets.map(lambda x: tokenizer(x['summary'], truncation=True), batched=True)['input_ids']
tokenized_target_len = [len(seq) for seq in tokenized_targets]
max_target_length = int(np.percentile(tokenized_target_len, 95))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/16369 [00:00<?, ? examples/s]

Max source length: 296


Map:   0%|          | 0/16369 [00:00<?, ? examples/s]

Max target length: 59


Tokenize dataset for summarization finetuning

In [7]:
def add_prefix(examples):
    """
    Add 'summarize: ' prefix for T5 to function properly
    """
    inputs = ["summarize: " + item for item in examples]
    return inputs


def tokenize_dataset(examples):
    """
    Tokenizes dataset splits and return data ready for finetuning
    """
    inputs = add_prefix(examples['dialogue'])

    model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True)
    # separate max_length for summaries (labels)
    labels = tokenizer(examples['summary'], max_length=max_target_length, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [8]:
tokenized_dataset = dataset.map(tokenize_dataset, batched=True, remove_columns=['id', 'dialogue', 'summary'])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})


Check encoded and decoded texts

In [10]:
for split in ['train', 'validation', 'test']:
    print(f'Tokenized text: {tokenized_dataset[split][0]["input_ids"]}')
    print(f'Decoded text: {tokenizer.decode(tokenized_dataset[split][0]["input_ids"], skip_special_tokens=True)}')

Tokenized text: [21603, 10, 21542, 10, 27, 13635, 5081, 5, 531, 25, 241, 128, 58, 16637, 10, 10625, 55, 21542, 10, 27, 31, 195, 830, 25, 5721, 3, 10, 18, 61, 1]
Decoded text: summarize: Amanda: I baked cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-)
Tokenized text: [21603, 10, 71, 10, 2018, 3059, 6, 33, 25, 3164, 5721, 22, 7, 3742, 58, 272, 10, 27, 22, 51, 1134, 417, 27, 183, 5, 363, 22, 7, 95, 58, 71, 10, 1072, 25, 281, 28, 140, 12, 8, 2586, 8596, 58, 5, 272, 10, 363, 103, 25, 241, 12, 103, 58, 71, 10, 27, 241, 12, 129, 3, 9, 17351, 21, 82, 520, 5, 272, 10, 466, 56, 143, 376, 78, 1095, 5, 71, 10, 11475, 6, 62, 22, 162, 5172, 34, 186, 648, 5, 27, 317, 3, 88, 22, 7, 1065, 230, 5, 272, 10, 466, 22, 7, 207, 5, 2922, 4890, 3, 9, 1782, 19, 3, 9, 3429, 962, 5, 2792, 578, 3, 9, 1871, 3, 117, 18, 61, 71, 10, 27, 31, 195, 129, 376, 80, 13, 273, 385, 3887, 5, 272, 10, 555, 24, 751, 31, 17, 1604, 95, 396, 600, 117, 18, 61, 71, 10, 275, 3, 1544, 396, 231, 117, 18, 61, 6

## Metrics

Create ROUGE metric and custom compute metrics function from [HuggingFace](https://huggingface.co/docs/transformers/tasks/summarization#evaluate)

In [11]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print(predictions, labels)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True,  clean_up_tokenization_spaces=True)
    # print(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [label.strip() for label in labels]

#     # rougeLSum expects newline after each sentence
#     preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
#     labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

#     return preds, labels


# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     # Replace -100s used for padding as we can't decode them
#     preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

# Finetuning [T5-small](https://huggingface.co/t5-small) model on [SAMSum Corpus ](https://huggingface.co/datasets/samsum)

## Modeling

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(CFG.model_path)

optim = AdamW(model.parameters())

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if CFG.fp16 else None,
)

NameError: ignored

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# config to set model generation settings
generation_config = GenerationConfig(
    max_new_tokens = CFG.max_summary_length,
    num_beams = 4,
    early_stopping = True,
    decoder_start_token_id = model.config.decoder_start_token_id
)

train_args = Seq2SeqTrainingArguments(
    output_dir = './t5-small-summarization',
    per_device_train_batch_size = CFG.per_device_batch_size,
    per_device_eval_batch_size = CFG.per_device_batch_size,
    num_train_epochs = CFG.num_epochs,
    warmup_steps = CFG.warmup_steps,
    fp16=CFG.fp16,
    do_train = True,
    do_eval = True,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = 'steps',
    save_total_limit=2,
    logging_strategy = 'steps',
    logging_steps = 100,
    dataloader_drop_last = True,
    # saves more VRAM if set to `False`
    dataloader_pin_memory = False,
    predict_with_generate = True,
    generation_config = generation_config
)

trainer = Seq2SeqTrainer(
    model = model,
    args = train_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    optimizers = (optim, None),
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.989400,1.834059,0.459700,0.221800,0.381500,0.381300,50.000000
1000,1.774300,1.754916,0.478900,0.236600,0.396600,0.395900,50.000000
1500,1.717700,1.708531,0.488400,0.242000,0.398100,0.397800,50.000000
2000,1.536800,1.697284,0.493100,0.244400,0.399400,0.399200,50.000000
2500,1.518700,1.670291,0.491900,0.242300,0.398200,0.397700,50.000000


TrainOutput(global_step=2760, training_loss=1.763051312211631, metrics={'train_runtime': 1055.4866, 'train_samples_per_second': 41.873, 'train_steps_per_second': 2.615, 'total_flos': 4556586265608192.0, 'train_loss': 1.763051312211631, 'epoch': 3.0})

# LoRA finetuning

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    CFG.model_path
    # load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(
    CFG.model_path
)

Create LoRA finetuning config

In [13]:
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none", # or 'lora_only'
    task_type=TaskType.SEQ_2_SEQ_LM
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9653981848502878


In [14]:
label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if CFG.fp16 else None,
)

optim = AdamW(model.parameters())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
# config to set model generation settings
generation_config = GenerationConfig(
    max_new_tokens = max_target_length,
    num_beams = 4,
    early_stopping = True,
    decoder_start_token_id = model.config.decoder_start_token_id
)

train_args = Seq2SeqTrainingArguments(
    output_dir = './t5-small-lora-summarization-7epochs',
    per_device_train_batch_size = CFG.per_device_batch_size,
    per_device_eval_batch_size = CFG.per_device_batch_size,
    num_train_epochs = CFG.num_epochs,
    warmup_steps = CFG.warmup_steps,
    fp16=CFG.fp16,
    do_train = True,
    do_eval = True,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = 'steps',
    logging_strategy = 'steps',
    logging_steps = 100,
    dataloader_drop_last = True,
    # saves more VRAM if set to `False`
    dataloader_pin_memory = True,
    predict_with_generate = True,
    generation_config = generation_config,
    # Saving best model to HF
    push_to_hub = True,
    hub_strategy = 'every_save',
    hub_model_id = 'NochnoyRitzar/t5-small-summarization-samsum',
    load_best_model_at_end = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = train_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    optimizers = (optim, None),
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.107400,1.922988,0.401100,0.182000,0.341800,0.341700,15.743900
1000,2.052600,1.855889,0.412200,0.184100,0.347800,0.348000,16.386000
1500,2.007500,1.819266,0.427300,0.195500,0.355200,0.355100,16.855400
2000,1.970000,1.808551,0.422200,0.192200,0.355100,0.355200,16.761000
2500,1.931000,1.790679,0.431800,0.200500,0.362900,0.362900,16.897100


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=2760, training_loss=2.0597116083338642, metrics={'train_runtime': 632.0315, 'train_samples_per_second': 69.927, 'train_steps_per_second': 4.367, 'total_flos': 3428432058777600.0, 'train_loss': 2.0597116083338642, 'epoch': 3.0})

# Prompt experiments on LoRA finetuned model

In [34]:
# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(CFG.model_path)
tokenizer = AutoTokenizer.from_pretrained(CFG.lora_model_hub_id)

# Load the LoRA model
model = PeftModel.from_pretrained(model, CFG.lora_model_hub_id)
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=512, out_features=512, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

In [16]:
# select random sample from test dataset
sample = dataset['test'].select([np.random.randint(0, len(dataset['test']))])

Generate summary using various methods: Greedy Search, Beam Search, Top-k Sampling, Top-p Sampling

In [38]:
print('----- Sampled dialogue -----')
print(sample['dialogue'][0], '\n\n')


input_ids = tokenizer('summarize: ' + sample['dialogue'][0], return_tensors='pt')['input_ids']


# Define the generation parameters
generation_params = {
    'max_length': 50,
    'num_beams': 1,
    'no_repeat_ngram_size': 2,
    'early_stopping': True,
    'do_sample': False,
    'top_k': None,
    'top_p': None,
    'temperature': 1.0,
}


# Generate using Greedy Search
generation_params['num_beams'] = 1
generation_params['do_sample'] = False
output_ids = model.generate(input_ids=input_ids, **generation_params)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Greedy Search:", output_text)

# Generate using Beam Search
generation_params['num_beams'] = 4
generation_params['do_sample'] = False
output_ids = model.generate(input_ids=input_ids, **generation_params)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Beam Search:", output_text)

# Generate using Top-k Sampling
generation_params['num_beams'] = 4
generation_params['do_sample'] = True
generation_params['top_k'] = 50
generation_params['top_p'] = 0.95
output_ids = model.generate(input_ids=input_ids, **generation_params)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Top-k Sampling:", output_text)

# Generate using Top-p Sampling
generation_params['num_beams'] = 4
generation_params['do_sample'] = True
generation_params['top_k'] = None
generation_params['top_p'] = 0.95
output_ids = model.generate(input_ids=input_ids, **generation_params)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Top-p Sampling:", output_text)

----- Sampled dialogue -----
Steph: <file_photo>
Steph: and what d'you think?
Sharon: hey! it is superb! I love the lamps in particular
Steph: ikea
Steph: wasn't sure about the colour scheme but the lamps decided
Sharon: copper is very much in
Steph: still need proper hangers, preferably from copper
Sharon: not too expensive?
Steph: not made of copper, but copper coloured
Sharon: but of course, stupid me
Sharon: where can you get them?
Steph: dunno! 


Greedy Search: Steph is looking for ikea lamps. Sharon is not sure about the colour scheme but they decided to buy copper.
Beam Search: Steph wants Sharon to buy ikea lamps.
Top-k Sampling: Steph wants Sharon to buy ikea lamps. They are not made of copper, but copper coloured.
Top-p Sampling: Steph wants Sharon to buy ikea lamps. They are not made of copper but copper coloured.


As with each run generation will be random, then it's hard to say which method was better, but contextually Top-k and Top-p sampling performed better than Greedy and Beam search

# Metrics results

I chose **ROUGE** and **BERTScore**

### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

**Pros**:
* **Multiple Variants:** It has multiple versions (ROUGE-1, ROUGE-2, ROUGE-L) that evaluate different aspects like unigram, bigram, and longest common subsequence.
* **Favorable for Recall:** Focuses on recall, which is important for capturing the essential information from the reference summaries.

**Cons**:
* **Insensitive to Synonyms and Paraphrasing:** ROUGE might not fully capture semantic meaning and can penalize paraphrased or synonymous words, leading to potentially lower scores for well-summarized content.
* **Doesn't Account for Fluency or Grammatical Correctness:** It measures overlap in content but doesn't consider fluency or grammatical correctness.

### BERTScore

**Pros**:
* **Leverages BERT Embeddings:** Uses contextual embeddings from BERT, which capture deeper semantic meaning and context compared to simple n-gram overlap metrics.
* **Addresses Semantic Similarity:** Better at capturing semantic similarity between sentences, which can better reflect the quality of summaries.
* **Correlates with Human Judgment:** Has shown strong correlations with human evaluation, indicating its effectiveness in capturing quality.

**Cons**:
* **Computationally Intensive:** Relies on pre-trained BERT embeddings, which can be computationally expensive and slow for large datasets.
* **Resource Dependency:** Requires the availability of the BERT model or its variants for evaluation, making it less portable compared to simpler metrics like ROUGE.

Generate prediction for whole test dataset

In [9]:
# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(CFG.model_path)
tokenizer = AutoTokenizer.from_pretrained(CFG.lora_model_hub_id)

# Load the LoRA model
model = PeftModel.from_pretrained(model, CFG.lora_model_hub_id).cuda()
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=512, out_features=512, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

In [11]:
from tqdm import tqdm

topk_generation_params = {
    'max_length': 50,
    'num_beams': 4,
    'no_repeat_ngram_size': 2,
    'early_stopping': True,
    'do_sample': True,
    'top_k': 50,
    'top_p': 0.95,
    'temperature': 1.0,
}


def evaluate_peft_model(sample):
    input_ids = tokenizer('summarize: ' + sample['dialogue'], return_tensors='pt', truncation=True)['input_ids'].cuda()
    # generate summary
    outputs = model.generate(input_ids=input_ids, **topk_generation_params)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)

    return prediction, sample['summary']


predictions, references = [] , []
for sample in tqdm(dataset['test']):
    p, l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

100%|██████████| 819/819 [07:21<00:00,  1.86it/s]


In [21]:
print(predictions[0])

Betty's number is Lemme. Amanda asks Larry to call her last time we were at the park together.


Evaluate generated summaries using **ROUGE, BLEU, BERTScore**

In [32]:
rouge_metric = evaluate.load('rouge')
bertscore_metric = evaluate.load('bertscore')

rouge_score = rouge_metric.compute(predictions=predictions, references=references)
bert_score = bertscore_metric.compute(predictions=predictions, references=references, lang='en', use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
print(rouge_score)
print(bert_score)

{'rouge1': 0.4361630433972852, 'rouge2': 0.1964703907260848, 'rougeL': 0.357730898787614, 'rougeLsum': 0.35757032656644305}
{'precision': [0.878675103187561, 0.9486502408981323, 0.9249770641326904, 0.907836377620697, 0.9266825318336487, 0.9284191131591797, 0.8900774717330933, 0.8950308561325073, 0.914776086807251, 0.9370169639587402, 0.9440954923629761, 0.957587480545044, 0.9071487188339233, 0.966733455657959, 0.8988602161407471, 0.890764594078064, 0.9923387765884399, 0.9494044780731201, 0.870682418346405, 0.9609925746917725, 0.8783204555511475, 0.9080016016960144, 0.907513439655304, 0.9429324269294739, 0.9353334307670593, 0.9293238520622253, 0.8879343271255493, 0.8758041858673096, 0.9299506545066833, 0.9169281125068665, 0.9154470562934875, 0.928909182548523, 0.9362456798553467, 0.9535595178604126, 0.9395772814750671, 0.9199880361557007, 0.9127403497695923, 0.9009356498718262, 0.9279576539993286, 0.8927584290504456, 0.887427806854248, 0.9399533867835999, 0.9141266942024231, 0.968688845

In [39]:
bert_score.pop('hashcode')
avg_bert_score = {f'bert_{k}': np.mean(v) for k, v in bert_score.items()}
print(avg_bert_score)

{'bert_precision': 0.9187966171056214, 'bert_recall': 0.9092045006443438, 'bert_f1': 0.9137931915138813}


In [41]:
rouge_score.update(avg_bert_score)

In [42]:
# Create a DataFrame
df = pd.DataFrame(rouge_score, index=['t5-small-lora'])

df.head()

,rouge1,rouge2,rougeL,rougeLsum,bert_precision,bert_recall,bert_f1
t5-small-lora,0.436163,0.19647,0.357731,0.35757,0.918797,0.909205,0.913793


## Conclusion

ROUGE Scores: Scores around 0.3 or higher, especially above 0.4, are often considered relatively good, indicating substantial overlap with reference summaries.

BERTScore Metrics: Scores close to 1.0 signify high semantic similarity between model-generated summaries and reference summaries.

So, given my results, finetuning T5-small using LoRA performed very good.